In [1]:
import json
import os
from io import BytesIO

import eyed3
import ffmpeg
import requests
from yt_dlp import YoutubeDL

In [ ]:
# Playlist
youtube_urls = "https://music.youtube.com/playlist?list=PLBRGkzFPYJTmMwAog3TsQL9tJCVzY9gq-"

options = {
    "listformats": True,
    "ignoreerrors": True,
    "cookiefile": "./outputs/cookies.txt",
}

with YoutubeDL(options) as ydl:
    info = ydl.extract_info(youtube_urls, download=False)

# with open("./outputs/data/playlists.json", "w") as f:
    # json.dump(info, f)

with open("./outputs/data/playlists.json", "r") as f:
    info = json.load(f)

In [4]:
len(info["entries"])

87

In [9]:
options = {
    # "listformats": True,
    "cookiefile": "./outputs/cookies.txt",
    "outtmpl": "./outputs/downloads/%(title)s.%(ext)s",  # Output file name
}

for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")
    
    output_file = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    options["outtmpl"] = output_file
    
    if os.path.exists(output_file):
        print(f"{i + 1}. {entry['title']} - EXISTED")
        continue
    print(f"{i + 1}. {entry['title']}")

    options["format"] = format_id
    with YoutubeDL(options) as ydl:
        ydl.download(entry["original_url"])

print("Download completed!")

1. None
2. Bay Bay
[youtube] Extracting URL: https://music.youtube.com/watch?v=aZRxQbKm4nI
[youtube] aZRxQbKm4nI: Downloading webpage
[youtube] aZRxQbKm4nI: Downloading tv client config
[youtube] aZRxQbKm4nI: Downloading tv player API JSON
[youtube] aZRxQbKm4nI: Downloading ios player API JSON
[youtube] aZRxQbKm4nI: Downloading m3u8 information
[info] aZRxQbKm4nI: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Bay Bay.webm
[download] 100% of    3.70MiB in 00:00:00 at 7.17MiB/s   
3. Coconerd
[youtube] Extracting URL: https://music.youtube.com/watch?v=2COb2cFlq1s
[youtube] 2COb2cFlq1s: Downloading webpage
[youtube] 2COb2cFlq1s: Downloading tv client config
[youtube] 2COb2cFlq1s: Downloading tv player API JSON
[youtube] 2COb2cFlq1s: Downloading ios player API JSON
[youtube] 2COb2cFlq1s: Downloading m3u8 information
[info] 2COb2cFlq1s: Downloading 1 format(s): 251
[download] Destination: ./outputs/downloads/Coconerd.webm
[download] 100% of    3.27MiB in 00:00:00 

In [5]:
not_exists = []
for i, entry in enumerate(info["entries"]):
    if not entry:
        print(f"{i + 1}. None")
        continue

    formats = [f for f in entry["formats"] if f["resolution"] == "audio only"]
    formats.sort(key=lambda x: x["tbr"] or 0, reverse=True)
    format_id = formats[0]["format_id"]

    title = entry["title"]
    title = title.replace("｜", "|")
    title = title.replace("：", ":")
    title = title.replace("＂", "\"",)
    title = title.replace("/", "-")

    fulltitle = entry["fulltitle"]
    fulltitle = fulltitle.replace("｜", "|")
    fulltitle = fulltitle.replace("：", ":")
    fulltitle = fulltitle.replace("＂", "\"",)
    fulltitle = fulltitle.replace("/", "-")

    webm = "./outputs/downloads/" + title + "." + formats[0]["audio_ext"]
    mp3 = "./outputs/thumbnails/" + fulltitle + "." + "mp3"
    # webm = webm.replace("|", "｜")
    # webm = webm.replace(":", "：")
    # webm = webm.replace("\"", "＂")
    # webm = webm.replace("/", "⧸")
    
    
    if not os.path.exists(webm):
        not_exists.append(webm)
        print(f"{i+1}. Not existed - ", webm)
        continue
    if os.path.exists(mp3):
        print(f"{i+1}. Existed - ", mp3)
        continue

    ffmpeg.input(webm).output(mp3, vn=None, acodec="libmp3lame", q=0).run()

    audiofile = eyed3.load(mp3)
    img_data = requests.get(entry["thumbnail"]).content
    audiofile.tag.images.set(3, img_data, "image/jpeg")
    audiofile.tag.artist = entry.get("artist", None)
    audiofile.tag.album = entry.get("album", None)
    audiofile.tag.title = entry.get("fulltitle", None)

    audiofile.tag.save()

    print(f"{i+1}. Thumbnail added to {mp3}")

1. None


ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavfor

2. Thumbnail added to ./outputs/thumbnails/Bay Bay.mp3


ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavfor

3. Thumbnail added to ./outputs/thumbnails/Coconerd.mp3
4. None
5. None
6. None


...

85. Thumbnail added to ./outputs/thumbnails/HUSTLANG TP Bank - Vũ Trụ HUSTLANG ( Sản phẩm thuộc bản quyền của HUSTLANG và TP Bank).mp3


ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavfor

86. Thumbnail added to ./outputs/thumbnails/Bức Tranh Chưa Hoàn Thiện.mp3


ffmpeg version 9c33b2f Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1627813612080/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavfor

87. Thumbnail added to ./outputs/thumbnails/Luffy-19.mp3


In [6]:
len(not_exists), not_exists

(0, [])